In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd

# Import your CRUD module
from AAC_CRUD import CRUD

###########################
# Data Manipulation / Model
###########################

# Replace with your MongoDB username and password
username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 32619
db_name = "AAC"
collection_name = "animals"

# Connect to MongoDB via CRUD Module
db = CRUD(username, password, host, port, db_name, collection_name)

# Query MongoDB for all records
df = pd.DataFrame.from_records(db.read({}))

# Drop the '_id' column to avoid issues with ObjectID data types in Dash tables
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################

# Add the image (Logo of Grazioso Salvare)
image_filename = 'grazioso_savare_logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('utf-8')

app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'height':'250px', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'}),
    ]),
    html.Center(html.B(html.H1('CS-340 Dashboard - Developed by Tanner Meininger'))),
    html.Hr(),

    # Adding RadioItems for interactive filtering
    html.Div([
        html.Label('Filter by Rescue Type'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'All'}
            ],
            value='All',  # Default value (Reset - no filter applied)
            labelStyle={'display': 'inline-block'}
        )
    ]),
    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        page_action='native',
        page_current=0,
        page_size=10,
        style_table={'height': '400px', 'overflowY': 'auto'},
        style_cell={'textAlign': 'left'}
    ),

    html.Br(),
    html.Hr(),

    # Centered Pie Chart and Map
    html.Div(className='row',
             style={'display': 'flex', 'justify-content': 'center'},
             children=[
                 html.Div(id='graph-id', className='col s12 m6'),
                 html.Div(id='map-id', className='col s12 m6')
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to filter interactive data table based on Rescue Type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    query = {}
    
    if filter_type == 'Water Rescue':
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'Disaster or Individual Tracking':
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }

    # Query MongoDB based on the selected filter
    filtered_df = pd.DataFrame.from_records(db.read(query))
    
    # Drop '_id' column if it exists
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)

    # Return the filtered data
    return filtered_df.to_dict('records')


# Callback to update the pie chart (Pie chart for dog breeds)
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return html.Div("No data available to display")

    # Convert the virtual data into a DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Check if data exists and the necessary columns are available
    if 'breed' in dff.columns:
        # Generate the pie chart for dog breeds
        fig = px.pie(dff, names='breed', title='Distribution of Dog Breeds')

        fig.update_layout(
            height=600,
            font=dict(size=18),
            legend=dict(
                title="Dog Breeds",
                orientation="v",
                yanchor="middle",
                y=0.5,
                xanchor="left",
                x=1.05,
                font=dict(size=16)
            )
        )
    else:
        return html.Div("No valid data for pie chart")

    return [
        dcc.Graph(figure=fig)
    ]


# This callback will highlight selected columns in the data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Callback to update the geolocation map based on the selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None or len(index) == 0:
        return html.Div("No animal selected.")

    dff = pd.DataFrame.from_dict(viewData)
    selected_row = index[0]  # We allow single row selection, so index is a list of length 1

    # Ensure the necessary latitude/longitude data is available
    if 'location_lat' in dff.columns and 'location_long' in dff.columns:
        lat = dff.iloc[selected_row]['location_lat']
        lon = dff.iloc[selected_row]['location_long']
        breed = dff.iloc[selected_row]['breed']
        name = dff.iloc[selected_row]['name']

        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[lat, lon], children=[
                    dl.Tooltip(breed),
                    dl.Popup([html.H1("Animal Name"), html.P(name)])
                ])
            ])
        ]
    return html.Div("Location data not available for selected animal.")


# Run the app server
app.run_server(debug=True)
